In [1]:

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

import sys
sys.path.append("../../")
from src.soporte_metricas import (
    obtener_metricas,
    comparar_arboles
)
pd.options.display.float_format = "{:,.2f}".format
# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../../datos/06_Modelo6/06_autorenew_target_encoded.csv",index_col=0)
df.sample()

,offerType,vehicleType,fuelType,gearbox,notRepairedDamage,kilometer,price,yearOfRegistration,gama,powerCV_robust_scaler
229710,"6,145.40","5,539.81","8,645.84","10,535.78","7,025.45","4,052.00","12,900.00","5,379.46","7,471.21","12,027.54"


# Decision Tree
- El árbol de decisiones se usa para modelos no lineales
- Vamos a probarlo a ver si mejora nuestras métricas

In [3]:
X = df.drop(columns="price")
y = df[["price"]]
X_train, X_test, y_train, y_test = train_test_split(X, # Todo menos la Variable Respuesta
                                                    y, # La variable Respuesta
                                                    train_size=0.7, # Como dividir los datos en este caso 80% train, 20% test
                                                    random_state=42, # Semilla, para que lo haga igual para cualquier persona
                                                    shuffle=True 
                                                    )

In [4]:
modelo_arbol = DecisionTreeRegressor()
modelo_arbol.fit(X_train,y_train)
y_pred_test = modelo_arbol.predict(X_test)
y_pred_train = modelo_arbol.predict(X_train)
df_metricas = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricas

,r2_score,MAE,MSE,RMSE
train,0.93,"1,052.11","4,097,608.57","2,024.26"
test,0.76,"1,742.53","12,799,807.33","3,577.68"


# Mejora sustancial! pero...
- Seguimos teniendo un error bastante alto.
- Y aquí tenemos un overfitting de manual
- Vamos a paralizar este mo

Con todos estos params podemos intentar conseguir algo mucho mejor

In [5]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [20, 30, 40, 50],
    'min_samples_leaf': [5, 10, 15, 20],
    'max_leaf_nodes': [10 , 20, 30, 40]
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbol = grid_search_arbol.best_estimator_

In [6]:
modelo_final_arbol.fit(X_train,y_train)
y_pred_test = modelo_final_arbol.predict(X_test)
y_pred_train = modelo_final_arbol.predict(X_train)
df_metricas = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricas

,r2_score,MAE,MSE,RMSE
train,0.67,"2,511.17","18,535,599.23","4,305.30"
test,0.66,"2,499.02","18,031,414.75","4,246.34"


In [7]:
modelo_final_arbol

DecisionTreeRegressor(max_depth=10, max_leaf_nodes=40, min_samples_leaf=5,
                      min_samples_split=20)

Ha tardado 49 Segundos

# Que nos dice
La primera iteración dice que el mejor resultado es el siguiente:
- max_depth = 10
- max_leaf_nodes = 40
- min_samples_leaf = 5
- min_samples_split = 20

Esto no significa que sea el mejor aún nos dice que:
- max_depth = Entre 5 y 10 y 15
- max_leaf_nodes = Entre 35 y 40 y 45
- min_samples_leaf = Entre 0 y 7
- min_samples_split = Entre 15 y 20 y 25

Así que toca nueva iteración probando entre esos rangos

In [8]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'max_leaf_nodes': [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45],
    'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10],
    'min_samples_split': [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]   
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbol2 = grid_search_arbol.best_estimator_


KeyboardInterrupt: 

In [ ]:
modelo_final_arbol2.fit(X_train,y_train)
y_pred_test = modelo_final_arbol2.predict(X_test)
y_pred_train = modelo_final_arbol2.predict(X_train)
df_metricas2 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricas2

,r2_score,MAE,MSE,RMSE
train,0.42,"3,102.95","93,294,234.93","9,658.89"
test,0.25,"3,275.65","307,575,781.75","17,537.84"


In [ ]:
modelo_final_arbol2

DecisionTreeRegressor(max_depth=9, max_leaf_nodes=45, min_samples_leaf=4,
                      min_samples_split=21)

tardó 25 minutos

# Resultados
- max_depth = 9
- max_leaf_nodes = 45
- min_samples_leaf = 4
- min_samples_split = 21
### Otra iteración para ver si podemos ver el numero de nodes


In [11]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [8, 9, 10],
    'max_leaf_nodes': [45,50,55,60],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [20, 21, 22]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv3 = grid_search_arbol.best_estimator_

In [12]:
modelo_final_arbolv3.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv3.predict(X_test)
y_pred_train = modelo_final_arbolv3.predict(X_train)
df_metricasv3 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv3

,r2_score,MAE,MSE,RMSE
train,0.44,"2,952.80","91,466,538.06","9,563.81"
test,0.25,"3,135.27","305,711,121.13","17,484.60"


In [13]:
modelo_final_arbolv3

DecisionTreeRegressor(max_depth=8, max_leaf_nodes=60, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 8
- max_leaf_nodes = 60
- min_samples_leaf = 4
- min_samples_split = 21

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [14]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [7, 8, 9],
    'max_leaf_nodes': [60,70,80,90,100],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv4 = grid_search_arbol.best_estimator_

In [15]:
modelo_final_arbolv4.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv4.predict(X_test)
y_pred_train = modelo_final_arbolv4.predict(X_train)
df_metricasv4 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv4

,r2_score,MAE,MSE,RMSE
train,0.46,"2,792.96","87,732,214.85","9,366.55"
test,0.27,"2,971.31","299,041,128.40","17,292.81"


In [16]:
modelo_final_arbolv4

DecisionTreeRegressor(max_depth=9, max_leaf_nodes=100, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 9
- max_leaf_nodes = 100 
- min_samples_leaf = 4
- min_samples_split = 21

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [20]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [7, 8, 9],
    'max_leaf_nodes': [100,200,300,400,500],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv5 = grid_search_arbol.best_estimator_

In [21]:
modelo_final_arbolv5.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv5.predict(X_test)
y_pred_train = modelo_final_arbolv5.predict(X_train)
df_metricasv5 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv5

,r2_score,MAE,MSE,RMSE
train,0.49,"2,498.77","83,385,864.38","9,131.59"
test,0.27,"2,698.13","297,417,081.08","17,245.78"


In [22]:
modelo_final_arbolv5

DecisionTreeRegressor(max_depth=9, max_leaf_nodes=400, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 9
- max_leaf_nodes = 400 
- min_samples_leaf = 4
- min_samples_split = 21

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [24]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [6, 7, 8, 9, 10, 11, 12, 13],
    'max_leaf_nodes': [350,400,450],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [20,21,22]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv6 = grid_search_arbol.best_estimator_

In [25]:
modelo_final_arbolv6.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv6.predict(X_test)
y_pred_train = modelo_final_arbolv6.predict(X_train)
df_metricasv6 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv6

,r2_score,MAE,MSE,RMSE
train,0.51,"2,398.11","78,748,308.20","8,874.02"
test,0.28,"2,612.19","294,796,693.35","17,169.64"


In [26]:
modelo_final_arbolv6

DecisionTreeRegressor(max_depth=13, max_leaf_nodes=450, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 13
- max_leaf_nodes = 450
- min_samples_leaf = 4
- min_samples_split = 21

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [27]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [13,20,25, 30],
    'max_leaf_nodes': [420,450,480],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv7 = grid_search_arbol.best_estimator_

In [28]:
modelo_final_arbolv7.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv7.predict(X_test)
y_pred_train = modelo_final_arbolv7.predict(X_train)
df_metricasv7 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv7

,r2_score,MAE,MSE,RMSE
train,0.52,"2,383.97","78,555,378.92","8,863.15"
test,0.28,"2,600.29","294,610,230.83","17,164.21"


In [29]:
modelo_final_arbolv7

DecisionTreeRegressor(max_depth=13, max_leaf_nodes=480, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 13
- max_leaf_nodes = 480
- min_samples_leaf = 4
- min_samples_split = 21

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [38]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [12, 13, 14,15,16],
    'max_leaf_nodes': [460,470,480,490,500,510],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv8 = grid_search_arbol.best_estimator_

In [39]:
modelo_final_arbolv8.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv8.predict(X_test)
y_pred_train = modelo_final_arbolv8.predict(X_train)
df_metricasv8 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv8

,r2_score,MAE,MSE,RMSE
train,0.52,"2,368.62","78,153,888.59","8,840.47"
test,0.28,"2,586.13","294,472,449.22","17,160.20"


In [40]:
modelo_final_arbolv8

DecisionTreeRegressor(max_depth=14, max_leaf_nodes=510, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 8
- max_leaf_nodes = 80 
- min_samples_leaf = 2
- min_samples_split = 14

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [44]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [12, 13, 14,15, 16,50],
    'max_leaf_nodes': [500,510,520,530,540,550,800,1000],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv9 = grid_search_arbol.best_estimator_

In [45]:
modelo_final_arbolv9.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv9.predict(X_test)
y_pred_train = modelo_final_arbolv9.predict(X_train)
df_metricasv9 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv9

,r2_score,MAE,MSE,RMSE
train,0.53,"2,217.94","76,142,670.77","8,725.98"
test,0.28,"2,468.06","293,797,198.62","17,140.51"


In [47]:
modelo_final_arbolv9

DecisionTreeRegressor(max_depth=16, max_leaf_nodes=1000, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 8
- max_leaf_nodes = 80 
- min_samples_leaf = 2
- min_samples_split = 14

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [61]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [20,30,40,50],
    'max_leaf_nodes': [6000,7000,8000,9000],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv10 = grid_search_arbol.best_estimator_

In [62]:
modelo_final_arbolv10.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv10.predict(X_test)
y_pred_train = modelo_final_arbolv10.predict(X_train)
df_metricasv10 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv10

,r2_score,MAE,MSE,RMSE
train,0.55,"1,839.39","73,086,541.77","8,549.07"
test,0.28,"2,266.96","292,941,690.10","17,115.54"


In [64]:
modelo_final_arbolv10

DecisionTreeRegressor(max_depth=40, max_leaf_nodes=8000, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 40
- max_leaf_nodes = 8000
- min_samples_leaf = 4
- min_samples_split = 21

### Nueva iteración
- Quiero comprobar max_depth y max_leaf_nodes

In [ ]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [20,30,40,50],
    'max_leaf_nodes': [7200,7300,7400,8000,8200,8300],
    'min_samples_leaf': [4],
    'min_samples_split': [21]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 5, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbolv11 = grid_search_arbol.best_estimator_

In [66]:
modelo_final_arbolv11.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv11.predict(X_test)
y_pred_train = modelo_final_arbolv11.predict(X_train)
df_metricasv11 = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasv11

,r2_score,MAE,MSE,RMSE
train,0.55,"1,839.39","73,086,541.77","8,549.07"
test,0.28,"2,267.46","292,954,548.27","17,115.92"


In [67]:
modelo_final_arbolv11

DecisionTreeRegressor(max_depth=30, max_leaf_nodes=8000, min_samples_leaf=4,
                      min_samples_split=21)

# Resultados
- max_depth = 40
- max_leaf_nodes = 8000
- min_samples_leaf = 4
- min_samples_split = 21



# Probemos el modelo final

In [129]:
modelo_final_arbolv11.fit(X_train,y_train)
y_pred_test = modelo_final_arbolv11.predict(X_test)
y_pred_train = modelo_final_arbolv11.predict(X_train)
df_metricasvfinal = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricasvfinal

,r2_score,MAE,MSE,RMSE
train,0.55,"1,839.39","73,086,541.77","8,549.07"
test,0.28,"2,267.44","292,947,501.97","17,115.71"


# Demasiado Overfit
- No soy capaz de reducir el árbol
- Hay demasiado overfit
- Veamos importancia columnas

In [130]:
pd.DataFrame(modelo_final_arbol.feature_importances_,columns=["Importancia"], index= X.columns).sort_values(by="Importancia",ascending=False)

,Importancia
powerCV_robust_scaler,0.37
kilometer,0.18
brand,0.17
vehicleType,0.10
yearOfRegistration,0.10
gearbox,0.05
fuelType,0.01
notRepairedDamage,0.01
offerType,0.00


# Está mejor distribuida la importancia

# Para el Modelo 3
- Eliminar columna brand
    - Es probable que al ser tantas categorías estemos siendo demasiado específicos
    - Queremos predecir precios en general, no por marca
    
- Manteniendo el cambio del Modelo 2 añadimos el eliminar columna "brand"